In [1]:
from tflite_runtime.interpreter import Interpreter
import cv2
from PIL import Image
import numpy as np
import time


In [11]:
model_path = "./Models/converted_model.tflite"
interpreter = Interpreter(model_path)
interpreter.allocate_tensors()
w,x,y,z = interpreter.get_input_details()[0]['shape']
print(w," ",x," ",y," ",z)

1   96   96   3


In [7]:
interpreter.invoke()

output_details = interpreter.get_output_details()[0]
print(output_details)

{'name': 'Identity', 'index': 14, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


In [2]:

def classify_image(interpreter, image, top_k=1):
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = image

  interpreter.invoke()
  output_details = interpreter.get_output_details()[0]
  output = np.squeeze(interpreter.get_tensor(output_details['index']))

  scale, zero_point = output_details['quantization']
  output = scale * (output - zero_point)

  ordered = np.argpartition(-output, top_k)
  return [(i, output[i]) for i in ordered[:top_k]][0]

In [3]:
model_path = "./Models/converted_model.tflite"
interpreter = Interpreter(model_path)
interpreter.allocate_tensors()
w,x,y,z = interpreter.get_input_details()[0]['shape']
# Load an image to be classified.
image = Image.open("./Images/8/8_0.jpg").convert('RGB').resize((y, x))

# Classify the image.
time1 = time.time()
label_id, prob = classify_image(interpreter, image)
time2 = time.time()
classification_time = np.round(time2-time1, 3)
print("Classificaiton Time =", classification_time, "seconds.")
print(label_id, prob)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


RuntimeError: There is at least 1 reference to internal data
      in the interpreter in the form of a numpy array or slice. Be sure to
      only hold the function returned from tensor() if you are using raw
      data access.

In [20]:
from tflite_runtime.interpreter import Interpreter 
from PIL import Image
import numpy as np
import time

# def load_labels(path): # Read the labels from the text file as a Python list.
#   with open(path, 'r') as f:
#     return [line.strip() for i, line in enumerate(f.readlines())]

def set_input_tensor(interpreter, image):
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = image

def classify_image(interpreter, image, top_k=1):
  set_input_tensor(interpreter, image)

  interpreter.invoke()
  output_details = interpreter.get_output_details()[0]
  output = np.squeeze(interpreter.get_tensor(output_details['index']))
  print("Output", output)
  # scale, zero_point = output_details['quantization']
  # output = scale * (output - zero_point)

  # ordered = np.argpartition(-output, 1)
  # return [(i, output[i]) for i in ordered[:top_k]][0]

data_folder = "./"

model_path = data_folder + "Models/converted_model.tflite"
#label_path = data_folder + "labels_mobilenet_quant_v1_224.txt"

interpreter = Interpreter(model_path)
print("Model Loaded Successfully.")

interpreter.allocate_tensors()
_, height, width, _ = interpreter.get_input_details()[0]['shape']
print("Image Shape (", width, ",", height, ")")

# Load an image to be classified.
image = Image.open(data_folder + "Images/9/9_36.jpg").convert('RGB').resize((width, height))
image.show()
# Classify the image.
# time1 = time.time()
classify_image(interpreter, image)
# time2 = time.time()
# classification_time = np.round(time2-time1, 3)
# print("Classificaiton Time =", classification_time, "seconds.")
# print(label_id, prob)

# Read class labels.
# labels = load_labels(label_path)

# Return the classification label of the image.
# classification_label = labels[label_id]
# print("Image Label is :", classification_label, ", with Accuracy :", np.round(prob*100, 2), "%.")

Model Loaded Successfully.
Image Shape ( 96 , 96 )
Output [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
